<a href="https://colab.research.google.com/github/HueyVault/study_NLPs/blob/main/quests/DeepLearnings/similarity_semantic_searchs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 로드

In [1]:
!pip install datasets -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-

In [2]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("./hankyung_content.csv")
df = pd.DataFrame(df)
dataset_hankyung = Dataset.from_pandas(df)

## 의미 검색 유사도

### 데이터 준비

In [3]:
dataset_hankyung

Dataset({
    features: ['_id', 'CONTENT'],
    num_rows: 20000
})

In [4]:
# 일부 데이터만 사용 약 1000행 정도
data_size = 1000
klue_mrc_dataset_train = dataset_hankyung.train_test_split(train_size=data_size,
                                                    shuffle=False)['train']
klue_mrc_dataset_train

Dataset({
    features: ['_id', 'CONTENT'],
    num_rows: 1000
})

In [5]:
klue_mrc_dataset_train[10]
klue_mrc_dataset_train.shape

(1000, 2)

### 데이터 임베딩

In [6]:
from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model_sentence

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [7]:
embeddings = model_sentence.encode(klue_mrc_dataset_train['CONTENT'])
embeddings

array([[-0.54614305, -0.651236  , -0.7971319 , ..., -0.02598986,
         0.19752634,  0.06856576],
       [-0.23833852,  0.08987761, -0.08520548, ..., -0.4095496 ,
         0.07906791,  0.1983188 ],
       [-0.3005725 , -0.7300937 , -0.20646016, ..., -0.6025831 ,
         0.27324158, -0.5417452 ],
       ...,
       [ 0.04072866, -0.46540147, -0.29249972, ..., -0.17280045,
        -0.21546994,  0.5861441 ],
       [-0.13167846, -0.44667745, -0.54369164, ..., -0.12652655,
        -0.39351586,  0.7744218 ],
       [ 0.066888  , -0.5269056 , -0.23755114, ..., -0.63587356,
         0.5368899 , -0.31884116]], dtype=float32)

In [8]:
embeddings.shape

(1000, 768)

### 검색 인덱스 생성 (KNN)

In [11]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.10.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 46.4 MB/s eta 0:00:00


In [12]:
import faiss # meta api : vector distance

index_KNN = faiss.IndexFlatL2(embeddings.shape[1]) # KNN 알고리즘 초기화, vocab 사이즈 만큼 vector 공간 할당
index_KNN

<faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f9a5580ac40> >

In [13]:
# 인덱스에 embedding 저장, 일종의 테이블 생성(메모리용 vectorDB 유사)
# 신규 단어 처리 어떻게? db는 로컬에 저장?
index_KNN.add(embeddings)

In [14]:
type(index_KNN),index_KNN

(faiss.swigfaiss_avx512.IndexFlatL2,
 <faiss.swigfaiss_avx512.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7f9a5580ac40> >)

In [15]:
## 활용
query = '이번 연도에는 언제 비가 많이 올까?'

embedding_query = model_sentence.encode([query])
embedding_query.shape

(1, 768)

In [16]:
distances, indices = index_KNN.search(embedding_query, 3)
distances, indices

(array([[310.8513 , 325.69952, 367.06134]], dtype=float32),
 array([[403, 381, 410]]))

In [18]:
for idx in indices[0]:
    print(klue_mrc_dataset_train['CONTENT'][idx]) # 학습 데이터가 많아야 성능 좋다. 꼭 의미가 안맞아도 거리 가까운게 높게 나온다
    print('--------------------------------')

때 이른 불볕더위가 기승을 부리는 가운데 올여름 폭염과 장마철 강수량이 예년을 뛰어넘을 것이라는 예보가 나오자 대형마트가 과일 등 농산물 수급 대책 마련에 나섰다.





                최근 기상청 기후예측모델에 따르면 올해 6~8월 기온이 평년보다 높을 확률은 91~94%, 7~8월 강수량이 평년보다 많거나 비슷할 확률은 80%로 나타났다. 비가 많이 오면 낙과 피해와 함께 과수가 물을 많이 머금어 당도가 떨어지고 크기도 작아진다. 이상 고온이 지속돼도 일소(햇볕 뎀 현상), 밀 증상(내부가 투명하게 변하는 것) 등으로 상품성이 떨어질 가능성이 높다.16일 유통업계에 따르면 이달 말부터 여름 제철 과일이 이상 기후 피해를 볼 가능성이 있는 것으로 알려졌다. 특히 자두, 복숭아, 포도, 참외 등 여름 제철 과일은 쉽게 상해 장기 저장이 어렵다는 점이 수급 불안정성을 높이는 요인이다. 한 대형마트 바이어는 “여름 대표 과일인 복숭아와 수박의 수급이 현재 비교적 안정적이지만 폭염과 폭우가 나타나면 빠르게 악화할 수 있어 안심할 수 없다”고 했다.대형마트들은 추석 과일 선물 세트의 대표 구성품인 사과와 배의 생육과 수급 여건도 예의 주시하고 있다. 최근 사과와 배에 주로 생기는 과수화상병이 중부 지역 과수 농가를 중심으로 빠르게 확산하고 있기 때문이다.대형마트들은 대표 작물의 작황 부진에 대비해 대책 마련에 들어갔다. 롯데마트는 산지를 다변화하고, 기후 피해가 적은 산지의 농산물을 분산 출하할 계획이다. 복숭아는 혹서기 주요 산지를 기존 영남(경북 청도, 경남 함안, 경북 경산)에서 충북, 전북 전주로 옮기기로 했다. 수박은 산지를 충북 음성과 전북 고창에 더해 강원 양구, 경북 봉화, 전북 무주 등 산간 지역·고지대로 확대했다.이마트는 장마 기간 부족한 일조량을 보완해 농산물 품질을 유지한다는 방침이다. 사과·자두나무 아래에 반사 필름을 깔아 흐린 날씨에도 열매가 골고루 빛을 받을 수 있도록 농가와 협력하고 있다. 비교적 오래 저장할 수 있는 작물

### 라마 api 벡터 활용

In [19]:
!pip install llama-index llama-index-embeddings-huggingface -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [20]:
from llama_index.core import VectorStoreIndex, Document

In [23]:
# llama에 맞는 데이터셋 형식 구성
text_list = dataset_hankyung[:100]['CONTENT']
# len(text_list), text_list[99]
documents = [ Document(text=text) for text in text_list] # llama_index 형식
len(documents), documents[99]

(100,
 Document(id_='7496ec80-d310-4fb1-8da7-5c1c0e137023', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='대차·대주 상환기간 90일·최대 12개월로 통일…불법공매도시 징역 가중 공매도 거래가 내년 3월 공매도 전산시스템 구축 이후 전면 재개되면 개인투자자와 기관투자자의 공매도 거래조건이 같아진다.  이에 따라 기존에 공매도 거래 시 개인이 기관·법인투자자보다 불리한 \'기울어진 운동장\'이 해소될 수 있을지 주목된다.  불법 공매도에 대한 제재는 강화돼 부당이득액이 커질수록 징역이 가중될 전망이다.  ◇기관·개인 거래조건 통일…대차·대주 상환기간 90일·총 12개월\n                    \n\n\n\n\n                    13일 민당정협의회에서 합의된 공매도 제도개선방안 최종안에 따르면 내년 3월 이후 공매도가 전면 재개되면 개인투자자가 공매도 거래시 기관·법인투자자보다 불리하지 않도록 공매도 거래조건이 통일된다.  기관투자자의 공매도 목적 대차거래, 개인투자자의 대주서비스는 상환기간은 90일, 연장 시 최대 12개월로 같아진다.  다만, 공매도 상장지수펀드(ETF) 설정 등 공매도 목적이 아닌 대차거래는 상환기간에 제한을 두지 않는다.  대차거래에서는 대여자가 요구할 경우 차입자인 기관·법인이 즉시 상환해야 하는 리콜제도가 유지되지만, 대주서비스에 있어서는 최소 90일의 상환기간이 차입자인 개인에게 보장된다.  현재 기관투자자는 대차거래 상환기간이 특별히 정해지지 않았던 반면, 개인투자자는 대주서비스 상환기간이 

In [24]:
# llama_index vectorizing
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embedding_model = HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embedding_model

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbedding(model_name='snunlp/KR-SBERT-V40K-klueNLI-augSTS', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f9a20478190>, num_workers=None, max_length=128, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [25]:
## index 화
index_llama = VectorStoreIndex.from_documents(documents,
                                 embed_model=embedding_model)
index_llama

In [28]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [29]:
# 활용
query_engine = index_llama.as_query_engine(llm=None) # default openai

In [30]:
query_text = f'이번 연도에는 언제 비가 많이 올까?'
response = query_engine.query(query_text)

In [31]:
response

Response(response='There is no information provided in the context regarding when it will rain heavily this year.', source_nodes=[NodeWithScore(node=TextNode(id_='b38e3039-c893-45af-bf0c-138888ca9ca8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7e7ae3bf-9e9c-4070-ae14-85cf386a21ed', node_type='4', metadata={}, hash='702d3b749c79cfe8d92e46a0a3de006dac92d77e83abe2fbc09a712fcbd21069'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d980d0c8-f131-4cbd-b7a0-cb962291bd65', node_type='1', metadata={}, hash='31219e8db05aa29b60150c9138eb6a0d220f5eff6d24ef7f54d06939c48b8948')}, metadata_template='{key}: {value}', metadata_separator='\n', text='소비자와 시장 모두에 좋은 소식"이라고 평가했다.  빌 애덤스 댈러스 코메리카 은행 수석 이코노미스트는 로이터통신에 "점도표에 나온 의견이 실제로 FOMC의 향후 금리 결정에 그대로 반영되는 것은 아니다"라고 말했다.  그러면서 "모든 지역 연방준비은행 총재들이 점도표에 점을 찍지만, 이들 중 일부만 FOMC 결정에 투표하며, 투표에 참여하는 연은 총재들은 금리 인하 의견을 더 잘 수용하는 경향이 